In [40]:
import pandas as pd
import plotly as plt
import plotly.express as px

In [41]:
churn_data = pd.read_csv('data/churn.csv')
churn_data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [42]:
#Столбцы таблицы (так удобнее, чем в markdown):
# RowNumber — номер строки таблицы (это лишняя информация, поэтому можете сразу от неё избавиться)
# CustomerId — идентификатор клиента
# Surname — фамилия клиента
# CreditScore — кредитный рейтинг клиента (чем он выше, тем больше клиент брал кредитов и возвращал их)
# Geography — страна клиента (банк международный)
# Gender — пол клиента
# Age — возраст клиента
# Tenure — сколько лет клиент пользуется услугами банка
# Balance — баланс на счетах клиента в банке
# NumOfProducts — количество услуг банка, которые приобрёл клиент
# HasCrCard — есть ли у клиента кредитная карта (1 — да, 0 — нет)
# IsActiveMember — есть ли у клиента статус активного клиента банка (1 — да, 0 — нет)
# EstimatedSalary — предполагаемая заработная плата клиента
# Exited — статус лояльности (1 — ушедший клиент, 0 — лояльный клиент)

In [43]:
# удаляем столбец с номерами
churn_data.drop(['RowNumber'], axis=1, inplace=True)

In [44]:
# категоризируем столбцы с несколькими значениями (от 2), чтобы на диаграммах 
# иметь дело не с цветовой шкалой, а с набором значений

# категоризируем столбец для пола
churn_data.Gender = churn_data.Gender.astype('category')
churn_data.Gender = churn_data.Gender.cat.rename_categories({
    'Female': 'Женщины',
    'Male': 'Мужчины' 
})

# категоризируем столбец для количества услуг с сохранением порядка
churn_data.NumOfProducts = churn_data.NumOfProducts.astype(
    pd.api.types.CategoricalDtype(
        categories=sorted(churn_data.NumOfProducts.unique()), 
        ordered=True
    )
)

# категоризируем столбец наличия кредитной карты
churn_data.HasCrCard = churn_data.HasCrCard.astype('category')
churn_data.HasCrCard = churn_data.HasCrCard.cat.rename_categories({
    0: 'Нет',
    1: 'Есть' 
})

# категоризируем столбец активности
churn_data.IsActiveMember = churn_data.IsActiveMember.astype('category')
churn_data.IsActiveMember = churn_data.IsActiveMember.cat.rename_categories({
    0: 'Неактивный',
    1: 'Активный' 
})

# категоризируем столбец признака лояльности (вышедший/не вышедший)
churn_data.Exited = churn_data.Exited.astype('category')
churn_data.Exited = churn_data.Exited.cat.rename_categories({
    0: 'Текущий',
    1: 'Вышедший' 
})

### Задание 9.1. 
Каково соотношение ушедших и лояльных клиентов? 

Покажите это на графике и дайте комментарий по соотношению.


In [45]:
# 9.1. 
# Каково соотношение ушедших и лояльных клиентов? 
# Покажите это на графике и дайте комментарий по соотношению.

# данные, которые неужно отобразить на диаграмме
bar_data = churn_data.Exited.value_counts()

#строим график
fig = px.bar(
    x=['Лояльные', 'Ушедшие'],
    y=bar_data.values,
    height=500, #высота
    orientation='v', #ориентация графика
    width=1000, #ширина
    labels = {'y' : ' Количество', 'x': 'Лояльность'},
    title='Отношение ушедших и лояльных клиентов', #заголовок
    text=[f'{100*x/bar_data.values.sum():.2f}%' for x in bar_data.values]
)

#отображаем его
fig.show()

На графике видно, что ушедших клиентов значительно меньше. Доля ушедших к лояльным клиентам - 1/4.

### Задание 9.2. 
Постройте график, показывающий распределение баланса пользователей, у которых на счету больше 2 500 долларов. 

Опишите распределение и сделайте выводы.

In [46]:
# пользователи с искомым балансом
from tracemalloc import start


u2500 = churn_data[(churn_data.Balance > 2500) & ~(churn_data.Balance.isna())]
print(u2500.shape[0], '/', churn_data.shape[0])

fig1 = px.histogram(
    u2500,
    x='Balance',
    color = 'Exited',
    nbins=3,
    title='Диаграмма процентного отношения числа пользователей с балансом >2500',
    labels={"Exited": "Лояльность", 'count': 'y', 'Balance': 'Баланс'}
)
fig1.show()

fig2 = px.histogram(
    u2500,
    x='Balance',
    color = 'Exited',
    nbins=30,
    title='Диаграмма процентного отношения числа пользователей с балансом >2500',
    labels={"Exited": "Лояльность", 'y': 'Количество', 'Balance': 'Баланс'}
)
fig2.show()

6383 / 10000


На основе полученных данных и диаграмм, можно сделать следующие выводы.
- соотношение лояльный/ушедший начинает меняться после 100к 
- - до 100_000 - 1/4-1/5,
- - 100к-110к - 1/3,
- - 110к-120к - 1/2.5,
- - 120к-130к - 1/3,
- - 130к-140к - 1/3,
- - 140к-150к - 1/3.5,
- - 150к-160к - 1/3.5,
- - 160к-170к - 1/4,
- - 170к-180к - 1/3.5,
- - 180к-200кк - 1/3,
- - счета с бОльшими балансами единичные, однако можем заметить, что среди клиентов с балансом >200к остальнось 15 из 34 человек.
- основная часть клиентов имеет баланс выше 2500,
- Среди клиентов, чей баланс более 2500, меньше одного процента с балансом >200000,
- Наиболее распростанены счета с балансом 100-200 тысяч,
- Примерно половина из выбранных счетов имеют на балансе 100-140 тысяч.

### Задание 9.3. 
Посмотрите на распределение баланса клиента в разрезе признака оттока. 

Как различаются суммы на накопительном счёте ушедших и лояльных клиентов? 

Подумайте и напишите, с чем это может быть связано, что может не устраивать ушедших клиентов в банке.

In [47]:
# Посмотрите на распределение баланса клиента в разрезе признака оттока. 

fig = px.box(
    data_frame=churn_data, #DataFrame
    y='Exited',
    x='Balance', 
    color='Exited',
    points='all',
    labels= {
        'Exited': 'Признак лояльности',
        'Balance': 'Значение баланса'
    },
    orientation='h',
    height=500, #высота
    width=1000, #ширина
    title='Распределение лояльных и ушедших клиентов' #заголовок
)
fig.show()


Выводы по распределнию:

Не смотря на то, что мы видим очень много клиентов с балансом до 100$, медианные значения говорят, что лояльные клиенты имеют на счетах до 100к\$, в то время как ушедшие имеют на счетах >100к\$.

Значит можем предположить, что в банке условия для клиентов с балансом >100k\$ более выгодные.

In [52]:
# определим соотношение лояльный/ушедший
def loyal_per_exited(series):
    cs = {'Текущий': 0, 'Вышедший': 0}
    vc = series.value_counts()
    for ind, val in zip(vc.index, vc.values):
        cs[ind] = val
    return (cs['Текущий'] - cs['Вышедший']) / (cs['Текущий'] + cs['Вышедший'])

In [49]:
# Для проверки этой гипотезы добавим в набор столбец 
# "группа по балансу" с шагом 50000\$ (крупный шаг для прикидки, 
# при желании можно исследовать каждую группу более детально). 
churn_data['BalanceGroup'] = churn_data.Balance // 50000
    
# сгруппируем клиентов по выделенным группам и добавим столбец соотношения
graph_data = churn_data.groupby('BalanceGroup')['Exited', ].aggregate(loyal_per_exited)
display(graph_data)
fig = px.bar(
    data_frame=graph_data,
    y='Exited',
    height=500, #высота
    width=1000, #ширина
    title='Отношение ушедших и лояльных клиентов', #заголовок
)
fig.show()

,Exited
BalanceGroup,
0.0,0.715060
1.0,0.602386
2.0,0.484595
3.0,0.561497
4.0,-0.090909
5.0,-1.000000


На построенной диаграмме видно, что интерес пользователей к банку падает при переходе между выделенными группами. Следовательно, можно сделать вывод, что условия для держателей счетов с Большими суммами являются менее выгодными.

**Однако** мы можем заключить, что для клиентов с нулевым балансом вообще не принципиально, какие условия в банке. Поэтому имеет смысл обработать данные без таких счетов.


In [50]:
# Посмотрите на распределение баланса клиента в разрезе признака оттока. 
without0 = churn_data.query('Balance > 0').copy()

fig = px.box(
    data_frame=without0, #DataFrame
    y='Exited',
    x='Balance', 
    color='Exited',
    points='all',
    labels= {
        'Exited': 'Признак лояльности',
        'Balance': 'Значение баланса'
    },
    orientation='h',
    height=500, #высота
    width=1000, #ширина
    title='Распределение лояльных и ушедших клиентов' #заголовок
)
fig.show()

При таком фильтре мы видим, что ушедшие клиенты имеют большую сконцентрированность вокруг медианы. Которая отличается среди групп на примерно 200\$. То есть наше предположение про невыгодные условия для клиентов с балансом >100k\$ не подтверждается.

Теперь посмотрим на коэффициент оттока без нулей.

In [109]:
# Для проверки этой гипотезы добавим в набор столбец 
# "группа по балансу" с шагом 50000\$ (крупный шаг для прикидки, 
# при желании можно исследовать каждую группу более детально). 
without0['BalanceGroup'] = without0.Balance // 50000
    
# сгруппируем клиентов по выделенным группам и добавим столбец соотношения
graph_data = without0.groupby('BalanceGroup')['Exited', ].aggregate(loyal_per_exited)

names = {str(x//50000): f'{x-50000+1}-{x}' for x in range(50_000-1, 300_000, 50_000)}
names['0'] = '<50 000'
names['5'] = '>250 000'


fig = px.bar(
    data_frame=graph_data,
    x=names,
    y='Exited',
    labels= {
        'x': 'Коэффициент лояльности',
        'BalanceGroup': 'Группа по размеру счета'
    },
    height=500, #высота
    width=1000, #ширина
    title='Отношение ушедших и лояльных клиентов', #заголовок
)

fig.show()

Теперь можно сказать, что наиболее привлекательным банк считается для категорий 50к-200к.

Уменьшим группы.

In [ ]:
step = 10_000
without0['BalanceGroup'] = without0.Balance // step

# определим соотношение лояльный/ушедший
def loyal_per_exited(series):
    cs = {0: 0, 1: 0}
    vc = series.value_counts()
    for ind, val in zip(vc.index, vc.values):
        cs[ind] = val
    return (cs[0] - cs[1]) / (cs[0] + cs[1])
    
# сгруппируем клиентов по выделенным группам и добавим столбец соотношения
graph_data = without0.groupby('BalanceGroup')['Exited', ].aggregate(loyal_per_exited)
names = {str(int(x)): f'{int(x)*step}-{(int(x)+1)*step - 1}' 
         for x in graph_data.index}
min_ind = str(int(graph_data.index.min()))
names[min_ind] = f'<{names[min_ind].split("-")[1]}'

last_ind = str(int(graph_data.index.max()))
names[last_ind] = f'>{names[last_ind].split("-")[0]}'


fig = px.bar(
    data_frame=graph_data,
    x=names,
    y='Exited',
    height=500, #высота
    width=1000, #ширина
    title='Отношение ушедших и лояльных клиентов', #заголовок
)
fig.show()

После анализа ненулевых счетов видим, что бОльшая доля лояльных клиентов имеют баланс 30к-200к. При этом клиенты, не входящие в эти группы, предпочитают прекращать пользоваться услугами банка.

### Задание 9.4. 
Посмотрите на распределение возраста в разрезе признака оттока. 

В какой группе больше потенциальных выбросов? 

На какую возрастную категорию клиентов стоит обратить внимание банку?

In [55]:
# Фильтр на вышедших
exited = churn_data.query('Exited == "Вышедший"')\
         .groupby('Age')['Exited', ].count()

# для первичного анализа построим столбчатую диаграмму со всеми возрастами 
# (без группировок)

fig = px.bar(
    data_frame=exited,
    y='Exited',
    height=500, #высота
    # orientation='v', #ориентация графика
    width=1000, #ширина
    labels = {'Exited' : ' Количество вышедних', 'Age': 'Возраст'},
    title='Количество вышедших клиентов в разрезе возраста', #заголовок
    #text=[f'{100*x/bar_data.values.sum():.2f}%' for x in bar_data.values]
)
fig.show()

На диаграмме видим превалирующую группу с возрастом 35-52.

Исходя из полученной диаграммы можем условно разбить клиентов на 5 возрастных категорий.

- 18-27
- 28-34
- 35-52
- 53-60
- 60+

Для менее строгого анализа можно использовать три группы:
- 18-34
- 35-52
- 53+

In [56]:
# Исследуем группы на выбросы

# добавим группы
def get_age_group(age):
    if age <= 27: return '18-27'
    if age <= 34: return '28-34'
    if age <= 52: return '35-52'
    if age <= 60: return '53-60'
    return '60+'

churn_data['AgeGroup'] = churn_data.Age.apply(get_age_group)
churn_data.sort_values('AgeGroup', inplace=True)

fig = px.box(
    data_frame=churn_data, #DataFrame
    y='AgeGroup',
    x='Balance', 
    # color='Exited',
    points='all',
    labels= {
        'AgeGroup': 'Возрастная группа',
        'Balance': 'Значение баланса'
    },
    orientation='h',
    height=500, #высота
    width=1000, #ширина
    title='Распределение лояльных и ушедших клиентов' #заголовок
)
fig.show()

Диаграмма явно показывает, что клиенты с нулевым балансом искажают результаты статистического анализа. Поэтому исключим из анализа счета с нулевым балансом.


In [59]:
# Фильтр на вышедших с ненулевым балансом
exited = without0.query('Exited == "Текущий"')\
         .groupby('Age')['Exited', ].count()

# для первичного анализа построим столбчатую диаграмму со всеми возрастами 
# (без группировок)

fig = px.bar(
    data_frame=exited,
    y='Exited',
    height=500, #высота
    width=1000, #ширина
    labels = {'Exited' : ' Количество вышедних', 'Age': 'Возраст'},
    title='Количество вышедших клиентов в разрезе возраста', #заголовок
)
fig.show()

Распределение вышедших в разрезе возраста не изменило тренд. Теперь проведем анализ на выбросы

In [60]:
without0['AgeGroup'] = without0.Age.apply(get_age_group)
without0.sort_values('AgeGroup', inplace=True)

fig = px.box(
    data_frame=without0, #DataFrame
    y='AgeGroup',
    x='Balance', 
    # color='Exited',
    points='all',
    labels= {
        'AgeGroup': 'Возрастная категория',
        'Balance': 'Значение баланса'
    },
    orientation='h',
    height=500, #высота
    width=1000, #ширина
    title='Распределение лояльных и ушедших клиентов' #заголовок
)
for line in fig.data:
    if line.name == '0':
        line.name = 'Лояльные'
    else:
        line.name = 'Ушедшие'
fig.show()

После анализа ненулевых счетов видим, что:
- наиболее "уходящая" аудитория (35-52) имеет наибольшее потенциальное количество выбросов,
- банку стоит обратить внимание на возрастную категорию 35-52.

### Задача 9.5. 

Постройте график, который показывает взаимосвязь кредитного рейтинга клиента и его предполагаемой зарплаты. Добавьте расцветку по признаку оттока клиентов. 

Какова взаимосвязь между признаками? 

Если не видите явной взаимосвязи, укажите это.

In [101]:
# построим диаграмму для всех клиентов
fig = px.scatter(churn_data, x="CreditScore", y="EstimatedSalary", color='Exited',
                 labels={
                    'Exited': 'Лояльность',
                    'EstimatedSalary': """Предполагаемый<br> уровень дохода""",
                    'CreditScore': 'Кредитный рейтинг'
                 },
                 title='Распределение клиентов по кредитному рейтингу относительно предполагаемого уровня дохода')
fig.show()

In [103]:
# построим диаграмму для клиентов с положительным балансом
fig = px.scatter(without0, x="CreditScore", y="EstimatedSalary", color='Exited',
                 labels={
                    'Exited': 'Лояльность',
                    'EstimatedSalary': """Предполагаемый<br> уровень дохода""",
                    'CreditScore': 'Кредитный рейтинг'
                 },
                 title='Распределение клиентов с положительным балансом<br>по кредитному рейтингу относительно предполагаемого уровня дохода')
fig.show()

Из полученных диаграмм можно сделать предположение, что уходят чаще люди с хорошей кредитной историей (смещение точек, соответствующих ушедшим клиентам, относительно начала координат).

При этом предполагаемый доход не оказывает значительного влияния на распределение. 

Проведем дополнительный анализ по возрастным группам.

In [104]:
# по всем клиентам
fig = px.scatter(churn_data, x="CreditScore", y="EstimatedSalary",
                 color='Exited', facet_row="AgeGroup", height=800,
                 labels={
                    'Exited': 'Лояльность',
                    'EstimatedSalary': """Предполагаемый<br> уровень дохода""",
                    'CreditScore': 'Кредитный рейтинг'
                 },
                 title='Распределение клиентов по кредитному рейтингу '
                       'относительно предполагаемого уровня дохода<br> '
                       '(с учетом возрастной группы)')
fig.show()

In [105]:
# по клиентам с полодительным балансом
fig = px.scatter(without0, x="CreditScore", y="EstimatedSalary",
                 color='Exited', facet_row="AgeGroup",height=800,
                 labels={
                    'Exited': 'Лояльность',
                    'EstimatedSalary': """Предполагаемый<br> уровень дохода""",
                    'CreditScore': 'Кредитный рейтинг'},
                 title='Распределение клиентов c ненулевым балансом<br>'
                       'по кредитному рейтингу '
                       'относительно предполагаемого уровня дохода<br> '
                       '(с учетом возрастной группы)')
fig.show()

После анализа можно сказать, что между исследуемыми признаками нет очевидной взаимосвязи.

После анализа ушедших/лояльных клиентов можно сказать, что возрастная группа 18-34 является наиболее лояльной.

### Задание 9.6. 

Кто чаще уходит, мужчины или женщины? 
Постройте график, который иллюстрирует это.

In [71]:
# данные, которые неужно отобразить на диаграмме
bar_data = churn_data.query('Exited == "Вышедший"').groupby('Gender').count()

#строим график
fig = px.bar(
    data_frame=bar_data,
    y='Exited',
    height=500, #высота
    orientation='v', #ориентация графика
    width=1000, #ширина
    labels = {'Exited' : ' Количество ушедших', 'Gender': 'Пол'},
    title='Количество ушедших клиентов в разрезе пола'
)

#отображаем его
fig.show()

Из диаграммы видно, что женщин ушло больше чем мужчин. Однако, проанализируем нормированные данные. Ведь количество женщин относительно количества мужчин могло быть значительно больше.

In [87]:
# данные, которые неужно отобразить на диаграмме
bar_data = churn_data.copy()
bar_data.Exited = bar_data.Exited.apply({'Текущий': 0,'Вышедший': 1}.get)\
                  .astype(int)
bar_data = bar_data.groupby('Gender').mean()

#строим график
fig = px.bar(
    data_frame=bar_data,
    y='Exited',
    height=500, #высота
    orientation='v', #ориентация графика
    width=1000, #ширина
    labels = {'Exited' : ' Количество ушедших', 'Gender': 'Пол'},
    title='Относительное количество ушедших клиентов в разрезе пола'
)

#отображаем его
fig.show()

Относительная разница между столбами на диаграмме значительно увеличилась. Значит можем заключить, что женщины более сконны сменить банк.

Проведем аналогичный анализ ненулевых счетов.

In [106]:
# данные, которые неужно отобразить на диаграмме
bar_data = without0.copy()
bar_data.Exited = bar_data.Exited.apply({'Текущий': 0,'Вышедший': 1}.get)\
                  .astype(int)
bar_data = bar_data.groupby('Gender').mean()

#строим график
fig = px.bar(
    data_frame=bar_data,
    y='Exited',
    height=500, #высота
    orientation='v', #ориентация графика
    width=1000, #ширина
    labels = {'Exited' : ' Количество ушедших', 'Gender': 'Пол'},
    title='Относительное количество ушедших клиентов в разрезе пола'
)

#отображаем его
fig.show()

Заметим, что оба пола при ненулевом балансе имеют бОльшую мотивацию сменить банк.

In [89]:
# Проведем анализ распределения в разрезе пола
without0['AgeGroup'] = without0.Age.apply(get_age_group)
without0.sort_values('AgeGroup', inplace=True)

fig = px.box(
    data_frame=without0, #DataFrame
    y='AgeGroup',
    x='Balance', 
    points='all',
    labels= {
        'AgeGroup': 'Возрастная категория',
        'Balance': 'Значение баланса'
    },
    orientation='h',
    height=1500, #высота
    width=1000, #ширина
    facet_row="Gender",
    title='Распределение лояльных и ушедших клиентов' #заголовок
)
fig.show()

Интересно, что у уходящих из банка женщин медианное значение баланса с возрастом растет. В то время как аналогичный показатель у мужчин с возрастом падает.

### Задание 9.7. 

Как отток клиентов зависит от числа приобретённых у банка услуг? 

Для ответа на этот вопрос постройте многоуровневую столбчатую диаграмму.

In [107]:
bar_data = churn_data.groupby(['Exited', 'NumOfProducts'])['Surname',].count().reset_index()

fig = px.bar(
    bar_data,
    x='Exited',
    y='Surname',
    color='NumOfProducts', 
    title="Количество клиентов в разрезе количества услуг и лояльности",
    labels={
        'Exited': 'Лояльность',
        'Surname': 'Количество',
        'NumOfProducts': 'Количество оказанных услуг'
    })
fig.show()

Вывод: большинство вышдших клиентов воспользовались только одной услугой в банке.

Наблюдения:
- среди лояльных (оставшихся) клиентов более распространено количество услуг 1 или 2
- 3 и 4 услугами в основном пользовались вышедшие клиенты.



In [108]:
# то же самое для ненулевых счетов
bar_data = without0.groupby(['Exited', 'NumOfProducts'])['Surname',].count().reset_index()

fig = px.bar(
    bar_data,
    x='Exited',
    y='Surname',
    color='NumOfProducts', 
    title="Количество клиентов в разрезе количества услуг и лояльности",
    labels={
        'Exited': 'Лояльность',
        'Surname': 'Количество',
        'NumOfProducts': 'Количество оказанных услуг'
    })
fig.show()

Интересно, что не ушедшие клиенты с положительным балансом не склонны пользоваться больше, чем одной услугой. 

Также, на основе большего присутствия категорий для 3 и 4 услуг у вышедших клиентов, можно предположить, что данные услуги, в числе прочих критериев, являются существенным фактором при принятии решиния уйти из банка.

Вероятно банку стоит сосредоточить внимание на наиболее распространенных услугах и сделать их более привлекательными для клиентов.

### Задание 9.8. 

Как влияет наличие статуса активного клиента на отток клиентов? 

Постройте диаграмму, иллюстрирующую это. 

Что бы вы предложили банку, чтобы уменьшить отток клиентов среди неактивных?

In [122]:
# диаграмма для всех клиентов
hist_data = churn_data.groupby("IsActiveMember")['Exited'].value_counts().reset_index()
display(hist_data)
fig = px.histogram(
    hist_data,
    x='IsActiveMember',
    y='Exited',
    color='level_1', 
    barnorm='percent',
    text_auto=True,
    title="Количество клиентов в разрезе активности и лояльности",
    labels={
        'Exited': 'Количество клиентов',
        'level_1': 'Лояльность',
        'IsActiveMember': 'Активность'
    })
fig.show()


,IsActiveMember,level_1,Exited
0,Неактивный,Текущий,3547
1,Неактивный,Вышедший,1302
2,Активный,Текущий,4416
3,Активный,Вышедший,735


In [123]:
# диаграмма для всех клиентов
hist_data = without0.groupby("IsActiveMember")['Exited']\
                   .value_counts().reset_index()
display(hist_data)
fig = px.histogram(
    bar_data,
    x='IsActiveMember',
    y='Exited',
    color='level_1', 
    barnorm='percent',
    text_auto=True,
    title="Количество клиентов с положительным балансом в разрезе активности и лояльности",
    labels={
        'Exited': 'Количество клиентов',
        'level_1': 'Лояльность',
        'IsActiveMember': 'Активность'
    })
fig.show()


,IsActiveMember,level_1,Exited
0,Неактивный,Текущий,2123
1,Неактивный,Вышедший,982
2,Активный,Текущий,2723
3,Активный,Вышедший,555


По полученным данным видим, что процент ушедших среди неактивных пользователей выше. Следовательно надо нативно привлекать клиента к активности.

Опять же исходя из описания не понятно, что подразумевается под активностью. Если движение средств и количество транзакций, то можно мотивировать тратить деньги или класть. Например, кэшбек или проценты на остаток.

При определенном уровне трат, выдавать бонусы. Если клиент видит вовлеченность банка в повышении его комфорта (даже если этот комфорт не перекрывает минусы), с большой вероятностью он будет более лоялен.

### Задание 9.9. 

В какой стране доля ушедших клиентов больше? Постройте тепловую картограмму, которая покажет это соотношение на карте мира. Предположите, с чем это может быть связано.

In [148]:
# По всем клиентам
prep_data = churn_data.copy()
prep_data['Exited'] = prep_data['Exited']\
    .apply({'Текущий':0, 'Вышедший': 1}.get).astype(int)
choropleth_data = prep_data.groupby('Geography')['Exited'].mean().reset_index()

#строим график
fig = px.choropleth(
    data_frame=choropleth_data, #DataFrame
    locations="Geography", #столбец с локациями
    locationmode = "country names", #режим сопоставления локаций с базой Plotly
    color="Exited", #от чего зависит цвет
    range_color=[0, 1], #диапазон цвета
    title='Процент клиентов в разрезе стран, прекративших пользоваться услугами банка', #заголовок
    width=800, #ширина
    height=500, #высота
    color_continuous_scale='Reds', #палитра цветов
    labels={'Exited': 'Процент<br>ушедших<br>клиентов'}
)

#отображаем график
fig.show()

In [147]:
# По всем клиентам
prep_data = without0.copy()
prep_data['Exited'] = prep_data['Exited']\
    .apply({'Текущий':0, 'Вышедший': 1}.get).astype(int)
choropleth_data = prep_data.groupby('Geography')['Exited'].mean().reset_index()

#строим график
fig = px.choropleth(
    data_frame=choropleth_data, #DataFrame
    locations="Geography", #столбец с локациями
    locationmode = "country names", #режим сопоставления локаций с базой Plotly
    color="Exited", #от чего зависит цвет
    range_color=[0, 1], #диапазон цвета
    title='Процент клиентов в разрезе стран, прекративших пользоваться услугами банка', #заголовок
    width=800, #ширина
    height=500, #высота
    color_continuous_scale='Reds', #палитра цветов
    labels={'Exited': 'Процент<br>ушедших<br>клиентов'}
)

#отображаем график
fig.show()

,Geography,Exited
0,France,0.182203
1,Germany,0.324432
2,Spain,0.195618


За немцами водится слава пунктуальности и щепетильности к деталям. Возможно предложения банка либо недостаточно четко сформулированы, либо не выполняются буква в букву (срываются сроки сделок, долгие переводы, большие очереди, некачественная работа call-центра). Французы и Итальянцы более "свободные" по темпераменту и, возможно, на какие-то мелочи закрывают глаза.

Так же возможно, что данное соотношение обосновано географическим расположением центрального офиса. То есть банк в первую очередь ориентирован на жителей Франции/Италии и на основе работы с этими странами осуществляет деятельность в Германии.

### Задание 9.10. 

Переведите числовой признак CreditScore в категориальный. Для этого воспользуйтесь функцией get_credit_score_cat(), которая приведена ниже. Примените её к столбцу CreditScore и создайте новый признак CreditScoreCat — категории кредитного рейтинга.

In [149]:
def get_credit_score_cat(credit_score):
    if credit_score >= 300 and credit_score < 500:
        return "Very_Poor"
    elif credit_score >= 500 and credit_score < 601:
        return "Poor"
    elif credit_score >= 601 and credit_score < 661:
        return "Fair"
    elif credit_score >= 661 and credit_score < 781:
        return "Good"
    elif credit_score >= 781 and credit_score < 851:
        return "Excellent"
    elif credit_score >= 851:
        return "Top"
    elif credit_score < 300:
        return "Deep"

Постройте сводную таблицу, строками которой являются категории кредитного рейтинга (CreditScoreCat), а столбцами — количество лет, в течение которых клиент пользуется услугами банка (Tenure). В ячейках сводной таблицы должно находиться среднее по признаку оттока (Exited) — доля ушедших пользователей.

На основе полученной сводной таблицы постройте тепловую карту с аннотацией. Найдите на тепловой карте категории клиентов, которые уходят чаще всего.

In [153]:
churn_data['CreditScoreCat'] = churn_data['CreditScore'].apply(get_credit_score_cat)
churn_data['ExitedV'] = churn_data['Exited']\
    .apply({'Текущий':0, 'Вышедший': 1}.get).astype(int)

pv = churn_data.pivot_table(
        values='ExitedV',
        aggfunc='mean',
        columns='Tenure',
        index='CreditScoreCat')
fig = px.imshow(pv, text_auto=True, aspect="auto")
fig.show()

In [154]:
without0['CreditScoreCat'] = without0['CreditScore'].apply(get_credit_score_cat)
without0['ExitedV'] = without0['Exited']\
    .apply({'Текущий':0, 'Вышедший': 1}.get).astype(int)

pv = without0.pivot_table(
        values='ExitedV',
        aggfunc='mean',
        columns='Tenure',
        index='CreditScoreCat')
fig = px.imshow(pv, text_auto=True, aspect="auto")
fig.show()

На основе полученных графиков можно сделать вывод, что клиенты предпочитают выходить:
- в первый год,
- все клиенты, кроме категории excellent, имеют тенденцию выходить после трех лет обслуживания,
- клиенты из категории Very_poor выходят после 4 и больше восьми лет обслуживания.
- клиенты из категории excellent выходят после 5-6 или после 9-10 лет обслуживания,
- остальные категории не имеют ярковыраженных тенденций и выходят равномерно, можно выелить 3, 5, 9 лет обслуживания.

Вероятно это связано со сроками действия вкладов или кредитных договоров.